# Training

In this section you can train the LSTM model with PinkFloyd sounds. Only polyphonic encoding is supported due to complexity of the songs.

## Train polyphony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
from pathlib import Path
# Load songs
paths = list(Path("../mukkeBude/songs/pinkfloyd").rglob("*.mid"))
print(f"Found {len(paths)} songs.")

encoded_songs = []
for path in paths:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

print(f"Songs encoded: {len(encoded_songs)}")
print(mapping.textify(encoded_songs))

Found 5 songs.


E:\Python\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:364: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
E:\Python\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:365: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,


Songs encoded: 17908
xxbos xxpad n69 d1 xxsep d1 n74 d1 n69 d1 xxsep d1 n78 d16 n74 d16 n69 d16 xxsep d22 n78 d6 n74 d6 n69 d6 xxsep d6 n79 d4 n71 d4 xxsep d4 n78 d4 n69 d4 xxsep d4 n76 d2 n69 d2 xxsep d2 n76 d10 n69 d10 xxsep d10 n74 d4 n69 d4 xxsep d4 n73 d2 n69 d2 xxsep d2 n73 d3 n69 d3 xxsep d3 n69 d1 xxsep d1 n64 d10 xxsep d10 n62 d2 xxsep d2 n62 d16 xxsep d16 n62 d8 xxsep d8 n78 d3 n69 d3 xxsep d3 n79 d3 n71 d3 xxsep d2 n78 d3 n69 d3 xxsep d3 n76 d11 n69 d11 xxsep d11 n74 d3 xxsep d2 n73 d3 xxsep d3 n69 d3 xxsep d3 n64 d3 xxsep d2 n62 d3 xxsep d3 n61 d3 xxsep d3 n59 d3 xxsep d2 n57 d3 xxsep d3 n67 d16 n60 d16 xxsep d19 n67 d3 n60 d3 xxsep d3 n66 d3 n60 d3 xxsep d2 n64 d3 n60 d3 xxsep d3 n62 d6 xxsep d6 n66 d1 xxsep d1 n64 d1 xxsep d1 n64 d6 n60 d6 xxsep d6 n62 d2 n59 d2 xxsep d2 n62 d8 n59 d8 xxsep d12 n62 d4 n59 d4 xxsep d4 n64 d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n64 d2 xxsep d2 n66 d6 n59 d6 xxsep d6 n66 d8 n59 d8 xxsep d8 n67 d7 n60 d7 xxsep d7 n

Each node is represented like this:
```
c  = n60
c# = n61
d  = n62
...
```

To represent the duration of each note, we will use "`d1, d2, d3, ...`". For example, if we want to represent a C note with a duration of 1/8, we will use "`n60 d2`". If we want to represent a C# note with a duration of 1/16, we will use "`n61 d1`".

To sepperate the notes which are played at the same time we will use "`xxsep`". For example, if we want to represent a C note with a duration of 1/16 and a C# note with a duration of 1/16 played at the same time, we will use "`n60 d1 n61 d1 xxsep`".

To sepperate each song there is "`xxbos`" in the dataset.

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/pinkfloyd_lstm_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=50, batch_size=16, tensorboard_callback=tensorboard_callback)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 300)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               570368    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 300)               77100     
                                                                 
Total params: 647,468
Trainable params: 647,468
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1116/1116 [==============================] - 12s 9ms/step - loss: 2.2841 - accuracy: 0.4333
Epoch 2/50
1116/1116 [==============================] - 9s 8ms/step - loss: 1.6933 -

In [ ]:
# Save model
model.save("PinkFloyd_polyphonie_lstm")

'E:\\Python\\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\\mukkeBude\\model\\preTrainedModels\\PinkFloyd_polyphonie_lstm.h5'

In [ ]:
# Show the logs
%tensorboard --logdir logs/pinkfloyd_lstm_polyphonie

UsageError: Line magic function `%tensorboard` not found.


# Generate music

In this section you can generate music with a pre trained LSTM model.

To open the midi file we recomend [musecore](https://musescore.org/de)

## Generate polyphony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS


# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
model = MukkeBudeLSTM.load(mapping, "PinkFloyd_polyphonie_lstm")

In [ ]:
# Create song
generated_song = model.generate("n69 d1 xxsep d1 n74 d1 n69 d1 xxsep d1 n78 d16 n74 d16 n69 d16 xxsep d22 n78 d6 n74 d6 n69 d6 xxsep d6 n79 d4 n71 d4 xxsep d4 n78 d4 n69 d4 xxsep d4 n76 d2 n69 d2 xxsep d2 n76 d10 n69 d10 xxsep d10 n74 d4 n69 d4 xxsep d4 n73 d2 n69 d2 xxsep d2 n73 d3 n69 d3 xxsep d3 n69 d1 xxsep d1 n64 d10 xxsep d10 n62 d2 xxsep d2 n62 d16 xxsep d16 n62 d8 xxsep d8 n78 d3 n69 d3 xxsep d3 n79 d3 n71 d3 xxsep d2 n78 d3 n69 d3 xxsep d3 n76 d11 n69 d11 xxsep d11 n74 d3 xxsep d2 n73 d3 xxsep d3 n69 d3 xxsep d3 n64 d3 xxsep d2 n62 d3 xxsep d3 n61 d3 xxsep d3 n59 d3 xxsep d2 n57 d3 xxsep d3 n67 d16 n60 d16 xxsep d19 n67 d3 n60 d3 xxsep d3 n66 d3 n60 d3 xxsep d2 n64 d3 n60 d3 xxsep d3 n62 d6 xxsep d6 n66 d1 xxsep d1 n64 d1 xxsep d1 n64 d6 n60 d6 xxsep d6 n62 d2 n59 d2 xxsep d2 n62 d8 n59 d8 xxsep d12 n62 d4 n59 d4 xxsep d4 n64 d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n59 d1 xxsep d1", 1000, temperature=0.5)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
# They should not be removed from the generated song
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))

1/1 [==============================] - 0s 22ms/step


In [ ]:
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=140, instrument=m21.instrument.ElectricGuitar())

print(generated_song)

n69 d1 xxsep d1 n74 d1 n69 d1 xxsep d1 n78 d16 n74 d16 n69 d16 xxsep d22 n78 d6 n74 d6 n69 d6 xxsep d6 n79 d4 n71 d4 xxsep d4 n78 d4 n69 d4 xxsep d4 n76 d2 n69 d2 xxsep d2 n76 d10 n69 d10 xxsep d10 n74 d4 n69 d4 xxsep d4 n73 d2 n69 d2 xxsep d2 n73 d3 n69 d3 xxsep d3 n69 d1 xxsep d1 n64 d10 xxsep d10 n62 d2 xxsep d2 n62 d16 xxsep d16 n62 d8 xxsep d8 n78 d3 n69 d3 xxsep d3 n79 d3 n71 d3 xxsep d2 n78 d3 n69 d3 xxsep d3 n76 d11 n69 d11 xxsep d11 n74 d3 xxsep d2 n73 d3 xxsep d3 n69 d3 xxsep d3 n64 d3 xxsep d2 n62 d3 xxsep d3 n61 d3 xxsep d3 n59 d3 xxsep d2 n57 d3 xxsep d3 n67 d16 n60 d16 xxsep d19 n67 d3 n60 d3 xxsep d3 n66 d3 n60 d3 xxsep d2 n64 d3 n60 d3 xxsep d3 n62 d6 xxsep d6 n66 d1 xxsep d1 n64 d1 xxsep d1 n64 d6 n60 d6 xxsep d6 n62 d2 n59 d2 xxsep d2 n62 d8 n59 d8 xxsep d12 n62 d4 n59 d4 xxsep d4 n64 d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n59 d1 xxsep d1 n62 d1 xxsep d1 n64 d2 xxsep d2 n66 d6 n59 d6 xxsep d6 n66 d8 n59 d8 xxsep d8 n67 d7 n60 d7 xxsep d7 n74 d1 xxsep d1 n79 d3 n67 d3 xxse

In [ ]:
path = Path("generated_song_pinkfloyd_lstm_poly.mid")
utils.write_midi(new_song, path)